In [1]:
# Import the required libraries
import pandas as pd
import numpy as np

In [2]:
# Load the data 
df = pd.read_csv('Iris.csv')
df=df.drop(columns=['Id'],axis=1)
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
# Define the calculate entropy function
def calculate_entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
                        for i in range(len(classes))])
    return entropy_value

In [4]:
# Define the calculate information gain function
def calculate_information_gain(dataset,feature,label): 
    # Calculate the dataset entropy
    dataset_entropy = calculate_entropy(dataset[label])   
    values,feat_counts= np.unique(dataset[feature],return_counts=True)
    
    # Calculate the weighted feature entropy                                # Call the calculate_entropy function
    weighted_feature_entropy = np.sum([(feat_counts[i]/np.sum(feat_counts))*calculate_entropy(dataset.where(dataset[feature]
                              ==values[i]).dropna()[label]) for i in range(len(values))])    
    feature_info_gain = dataset_entropy - weighted_feature_entropy
    return feature_info_gain

In [5]:
# Set the features and label
features = df.columns[:-1]
label = 'Species'
parent=None
features

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object')

In [6]:
#Define the create decision tree function
def create_decision_tree(dataset, df, features, label, parent):
    datum = np.unique(df[label],return_counts=True)
    unique_data = np.unique(dataset[label])
    if len(unique_data) == 1:
        return unique_data[0]
    elif len(dataset) == 0:
        return unique_data[np.argmax(datum[1])]
    elif len(features) == 0:
        return parent
    else:
        parent = unique_data[np.argmax(datum[1])]
        # Call the calculate_information _gain function
        item_values = [calculate_information_gain (dataset, feature, label) for feature in features]
        optimum_feature_index = np.argmax(item_values)
        optimum_feature = features[optimum_feature_index]
        decision_tree = {optimum_feature:{}}
        features = [i for i in features if i != optimum_feature] 
        for value in np.unique (dataset [optimum_feature]):
            min_data = dataset. where(dataset[optimum_feature] == value).dropna()
            # Recusrsive call to create_decision_tree function
            min_tree = create_decision_tree(min_data, df,features, label, parent)
            decision_tree[optimum_feature][value] = min_tree
        return(decision_tree)

In [7]:
# Train the decision tree model
decision_tree = create_decision_tree(df,df,features,label,parent)

In [8]:
def predict(test_data,decision_tree):
    for nodes in decision_tree.keys():
        value = test_data[nodes]
        decision_tree = decision_tree[nodes][value]
        
        prediction = 0
        if type(decision_tree) is dict:
            prediction = predict(test_data,decision_tree)
        else:
            prediction = decision_tree
            break
    return prediction

In [9]:
# Predict using the trained model
sample_data = {'SepalLengthCm':5.1,'SepalWidthCm':3.5,'PetalLengthCm':1.4,'PetalWidthCm':0.2}
test_data = pd.Series(sample_data)
prediction = predict(test_data,decision_tree)
prediction

'Iris-setosa'